# SQL

**Цель** -  проанализировать базу данных

В базе данных информация о книгах, издательствах, авторах и пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**Подключение к базе данных**

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

**Обзор таблиц**

In [2]:
query =('''
    
     SELECT *
     FROM books
     LIMIT 1

''')

pd.io.sql.read_sql(query, con = engine) 

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93


In [3]:
query =('''
    
     SELECT *
     FROM authors
     LIMIT 1

''')

pd.io.sql.read_sql(query, con = engine) 

,author_id,author
0,1,A.S. Byatt


In [4]:
query =('''
    
     SELECT *
     FROM publishers
     LIMIT 1

''')

pd.io.sql.read_sql(query, con = engine) 

,publisher_id,publisher
0,1,Ace


In [5]:
query =('''
    
     SELECT *
     FROM ratings
     LIMIT 1

''')

pd.io.sql.read_sql(query, con = engine) 

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4


In [6]:
query =('''
    
     SELECT *
     FROM reviews
     LIMIT 1

''')

pd.io.sql.read_sql(query, con = engine) 

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...


**Задание 1**

Посчитайте, сколько книг вышло после 1 января 2000 года;

In [7]:
query =('''
    
     SELECT COUNT(book_id)
     FROM books
     WHERE publication_date > '2000-01-01'

''')

pd.io.sql.read_sql(query, con = engine) 

,count
0,819


После 1 января 2000 года вышло 819 книг

**Задание 2**

Для каждой книги посчитайте количество обзоров и среднюю оценку

In [13]:
query =('''
        SELECT a.title,
               a.count_review,
               b.avg_rating
        FROM 
             (SELECT a.book_id,
                     a.title,
                     COUNT(b.review_id) AS count_review
               FROM books AS a
               LEFT JOIN reviews AS b ON a.book_id = b.book_id
               GROUP BY a.book_id) AS a
        LEFT JOIN (SELECT a.book_id,
                          a.title,
                          ROUND(AVG(b.rating),1) AS avg_rating
                   FROM books AS a
                   LEFT JOIN ratings AS b ON a.book_id = b.book_id
                   GROUP BY a.book_id) AS b ON a.book_id = b.book_id
        ORDER BY a.count_review DESC

''')

pd.io.sql.read_sql(query, con = engine) 

,title,count_review,avg_rating
0,Twilight (Twilight #1),7,3.7
1,The Curious Incident of the Dog in the Night-Time,6,4.1
2,Harry Potter and the Chamber of Secrets (Harry...,6,4.3
3,The Da Vinci Code (Robert Langdon #2),6,3.8
4,The Glass Castle,6,4.2
...,...,...,...
995,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.0
996,The Natural Way to Draw,0,3.0
997,Essential Tales and Poems,0,4.0
998,Leonardo's Notebooks,0,4.0


**Задание 3**

Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [9]:
query =('''
    
     SELECT a.publisher,
            COUNT(b.book_id)
     FROM publishers AS a
     FULL OUTER JOIN books AS b ON a.publisher_id = b.publisher_id
     WHERE (b.num_pages > 50) 
     GROUP BY a.publisher_id  
     ORDER BY COUNT(b.book_id) DESC
     LIMIT 1;
     
     

''')

pd.io.sql.read_sql(query, con = engine) 

,publisher,count
0,Penguin Books,42


Издательство Penguin Books  - 42 книги толще 50 страниц

**Задание 4**

Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [10]:
query =('''

     SELECT d.author
     FROM
         ( SELECT c.book_id AS book
          FROM books AS b
          LEFT JOIN ratings AS c ON b.book_id = c.book_id
          GROUP BY c.book_id
          HAVING COUNT(c.rating_id) >= 50 ) AS a
     LEFT JOIN ratings AS b ON a.book = b.book_id
     LEFT JOIN books AS c ON a.book = c.book_id
     LEFT JOIN authors AS d ON c.author_id = d.author_id
     GROUP BY d.author_id
     ORDER BY AVG(b.rating) DESC
     LIMIT 1;
     
     
''')

pd.io.sql.read_sql(query, con = engine) 

,author
0,J.K. Rowling/Mary GrandPré


J.K. Rowling/Mary GrandPré - автор с самой высокой средней оценкой книг (50 и более оценок)

**Задание 5**

Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [11]:
query =('''

SELECT ROUND(AVG(a.count_review), 1)
FROM (SELECT username,
              COUNT(review_id) AS count_review
       FROM reviews 
       WHERE username IN (SELECT username
                          FROM ratings
                          GROUP BY username
                          HAVING COUNT(rating_id) > 50)
       GROUP BY username) AS a
       

''')

pd.io.sql.read_sql(query, con = engine)

,round
0,24.3


Среднее количество обзоров от пользователей, которые поставили больше 50 оценок составляет 24,3 обзоров на пользователя